# Introducing Jupyter Notebooks in Sphinx

This notebook showcases very basic functionality of rendering your jupyter notebooks as tutorials inside your sphinx documentation.

As part of the LINCC Frameworks python project template, your notebooks will be executed AND rendered at document build time.

You can read more about Sphinx, ReadTheDocs, and building notebooks in [LINCC's documentation](https://lincc-ppt.readthedocs.io/en/latest/practices/sphinx.html)

In [ ]:
import numpy as np
import pandas as pd
from pandas_ts.packer import pack_flat, pack_dfs


# Adopted from
# https://github.com/lincc-frameworks/tape/blob/6a694c4c138aadb1508c2a96de4fa63f90319331/tests/tape_tests/conftest.py#L15
def create_test_rows():
    num_points = 1000
    all_bands = np.array(["g", "r", "i", "z"])

    rows = {
        "id": 8000 + (np.arange(num_points) % 5),
        "time": np.arange(num_points),
        "flux": np.arange(num_points) % len(all_bands),
        "band": np.repeat(all_bands, num_points / len(all_bands)),
        "err": 0.1 * (np.arange(num_points) % 10),
        "count": np.arange(num_points),
        # Not sure that I'm ready for Nones
        # "something_else": np.full(num_points, None),
    }

    return rows


sources = pd.DataFrame(create_test_rows())
sources.set_index("id", inplace=True)
sources

In [ ]:
packed = pack_flat(sources, name="sources")
packed

### Single item of the packed series is returned as a new DataFrame

In [ ]:
packed.iloc[0]

In [ ]:
# Get the linearly interpolated flux for time=10
packed.apply(lambda df: np.interp(10.0, df["time"], df["flux"]))

### Get packed sources series and play with `.ts` accessor
This series is a collection of structures, each structure consist of multiple fields, and each field is a "list" of values. 

In [ ]:
packed.ts.to_flat()

In [ ]:
packed.ts.to_lists()

In [ ]:
packed.ts["flux"]

In [ ]:
packed.ts[["time", "flux"]]

In [ ]:
packed.dtype

### Modify underlying fields with `.ts` accessor

In [ ]:
# Change flux in place with flat arrays
packed.ts["flux"] = -1 * packed.ts["flux"].list.flatten()
packed.ts["flux"]

In [ ]:
# Change errors for object 8003
light_curve = packed.loc[8003]
light_curve["err"] += 25
# packed.lpc[8003] = ... does not work
packed.iloc[3:4] = [light_curve]
packed.iloc[0]

In [ ]:
# Delete field and add new one
del packed.ts["count"]
filters = packed.ts.delete_field("band")
filters = "lsst_" + filters.list.flatten()
packed.ts["filters"] = filters
packed

### Change all items and pack to a new Series

In [ ]:
# Subsample light curves
dfs = packed.apply(lambda df: df.iloc[::50])
subsampled = pack_dfs(dfs, name="subsampled")
packed.loc[8000], subsampled.loc[8000]